In [1]:
import gym
from gym import spaces
from kaggle_environments.envs.hungry_geese.hungry_geese import Observation,\
Configuration, Action, row_col, adjacent_positions, translate, min_distance,random_agent, GreedyAgent
import os
import numpy as np
from kaggle_environments import make
import random
import tensorflow as tf
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory
from tensorflow.keras.optimizers import Adam

Loading environment football failed: No module named 'gfootball'


In [2]:
env = make('hungry_geese')
trainer = env.train([None, 'greedy', 'greedy', 'greedy'])
episodes = 10
for i in range(episodes):
    state = env.reset(num_agents = 4)
    done = False
    score = 0
#    env.render(mode="ipython", width=500, height=450)
    while not done:
        legend = {0:'NORTH',1:'EAST',2:'SOUTH',3:'WEST'}
        action = random.randint(0,3)
        obs,reward, done, info = trainer.step(legend[action])
        score +=reward
    print('episode: ',i,'reward: ',score)

episode:  0 reward:  1402
episode:  1 reward:  501
episode:  2 reward:  1301
episode:  3 reward:  201
episode:  4 reward:  1501
episode:  5 reward:  301
episode:  6 reward:  201
episode:  7 reward:  801
episode:  8 reward:  501
episode:  9 reward:  301


In [9]:
def DQNet(obs,action,shape):
    model = tf.keras.Sequential()
    if shape == 5:
        model.add(tf.keras.layers.Reshape(obs, input_shape=(1,7,11,21)))
        
    model.add(tf.keras.Input(shape=obs))
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=(3,5),strides = 1,activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=1,strides = 1, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=(3,5),strides = 1, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=1,strides = 1, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3,strides = 1, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=1,strides = 1, activation='relu'))
    model.add(tf.keras.layers.BatchNormalization())
#     model.add(tf.keras.layers.GlobalAveragePooling2D())
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dense(32, activation='relu'))
    model.add(tf.keras.layers.Dense(8, activation='relu'))
    model.add(tf.keras.layers.Dense(action, activation='linear'))
    
#    model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
    
    return model
model = DQNet([7,11,21],4,5)
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_1 (Reshape)          (None, 7, 11, 21)         0         
_________________________________________________________________
input_2 (InputLayer)         multiple                  0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 5, 7, 32)          10112     
_________________________________________________________________
batch_normalization_6 (Batch (None, 5, 7, 32)          128       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 5, 7, 32)          1056      
_________________________________________________________________
batch_normalization_7 (Batch (None, 5, 7, 32)          128       
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 3, 64)         

In [4]:
place = 0
def get_nearest_cells(x,y):
    # returns all cells reachable from the current one
        result = []
        for i in (-1,+1):
            result.append(((x+i+7)%7, y))
            result.append((x,(y+i+11)%11))
        return result
def change_reward(done,agent_value,obs,prev_action,prev_food):
    global place
    try:
        observation = obs[-1]['observation']
    except:
        observation = obs[-1]
    player_index = observation['index']
    rewards = [500,200,80,0]
    if len(observation['geese'][player_index]) == 0:
        alive = [i for i in observation['geese'] if len(i)>0]
        place = len(alive)
    if done:
        reward = rewards[place]
        place = 0
        return reward
    else:
        player_pos =  observation['geese'][player_index][0]
        if player_pos in prev_food:
            return 5
        else:
            return 1
            
    

In [5]:
prev_action = None
prev_food = []
class HungryGeeseEnv(gym.Env):
    def __init__(self):
        self.env = make("hungry_geese")
        self.config = self.env.configuration
        self.action_space = spaces.Discrete(4)
        self.observation_space = spaces.Box(low=-1,
                                            high=1, 
                                            shape=(self.config.rows, 
                                                   self.config.columns, 
                                                   1), 
                                            dtype=np.float64)
        self.trainer = self.env.train([None,'greedy','greedy','greedy'])
        self.NUM_AGENTS = 4
    def transform_obs(self,obs_dict, config_dict):
        try:
            observation = obs_dict[-1]['observation']
        except:
            observation = obs_dict[-1]
        food_pos = []
        player = observation['index']
        final_table = np.zeros((21, 7 * 11), dtype=np.float32)
        for p, geese in enumerate(observation['geese']):
            # head position
            for pos in geese[:1]:
                final_table[0 + (p - player) % self.NUM_AGENTS, pos] = 1
            # tip position
            for pos in geese[-1:]:
                final_table[4 + (p - player) % self.NUM_AGENTS, pos] = 1
            # whole position
            for pos in geese:
                final_table[8 + (p - player) % self.NUM_AGENTS, pos] = 1
            try:    
                x,y = row_col(geese[0], 11)
                possible_moves = get_nearest_cells(x,y) 

                for i in possible_moves:
                    pos = x*11 + y
                    final_table[17 + (p - player) % self.NUM_AGENTS,pos] = 1 
            except:
                pass

        # previous head position
        if len(geese)>0:
                x,y = row_col(geese[0], 11)
                possible_moves = get_nearest_cells(x,y) 

                for i in possible_moves:
                    pos = x*11 + y
                    final_table[17 + (p - player) % self.NUM_AGENTS,pos] = 1 

        # food
        for pos in observation['food']:
            final_table[16, pos] = 1
            food_pos.append(pos)
        return np.transpose(final_table.reshape(-1,7,11),(1,2,0)),food_pos
    
    def reset(self):
        self.obs = []
        self.obs.append(self.env.reset(num_agents = 4)[0])
        self.transformed_obs,food_data = self.transform_obs(self.obs,self.config)
        return self.transformed_obs
    
    def transform_action(self,agent_value):
        # We need to transform the numerical values retruned by the algorithm 
        #into string values accepted by the default env 
        if agent_value == 0:
            return "NORTH"
        if agent_value == 1:
            return "EAST"
        if agent_value == 2:
            return "WEST"
        if agent_value == 3:
            return "SOUTH"
    
    def step(self, agent_value):
        global prev_action
        global prev_food
        food_data = []
        action = self.transform_action(agent_value)
        current_obs, reward, done, info = self.trainer.step(action)
        self.obs.append(current_obs)
        if prev_action is None:
            prev_action = agent_value
        if done == False:
            self.transformed_obs,food_data = self.transform_obs(self.obs, self.config)
        reward = change_reward(done,agent_value,self.obs,prev_action,prev_food)
        prev_food = food_data
        prev_action = action
        return self.transformed_obs, reward, done, info
env = HungryGeeseEnv()

In [6]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit = 1000000, window_length = 1)
    dqn = DQNAgent(model = model, memory = memory, policy = policy,nb_actions = 4, nb_steps_warmup = 100, target_model_update = 1e-2)
    return dqn

In [10]:
dqn = build_agent(model,4)
dqn.compile(Adam(lr = 1e-3,clipvalue=1), metrics = ['mae'])
dqn.fit(env, nb_steps = 50000, visualize = False, verbose = 2)

Training for 50000 steps ...


/home/hsyed/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


     6/50000: episode: 1, duration: 2.073s, episode steps:   6, steps per second:   3, episode reward:  0.500, mean reward:  0.083 [ 0.000,  0.100], mean action: 1.167 [0.000, 2.000],  loss: --, mae: --, mean_q: --
     9/50000: episode: 2, duration: 0.023s, episode steps:   3, steps per second: 129, episode reward:  0.200, mean reward:  0.067 [ 0.000,  0.100], mean action: 1.000 [0.000, 3.000],  loss: --, mae: --, mean_q: --
    17/50000: episode: 3, duration: 0.042s, episode steps:   8, steps per second: 192, episode reward:  0.700, mean reward:  0.088 [ 0.000,  0.100], mean action: 1.875 [0.000, 3.000],  loss: --, mae: --, mean_q: --
    21/50000: episode: 4, duration: 0.025s, episode steps:   4, steps per second: 160, episode reward:  0.300, mean reward:  0.075 [ 0.000,  0.100], mean action: 1.250 [0.000, 2.000],  loss: --, mae: --, mean_q: --
    27/50000: episode: 5, duration: 0.033s, episode steps:   6, steps per second: 184, episode reward:  0.500, mean reward:  0.083 [ 0.000, 

/home/hsyed/anaconda3/envs/tf2/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


   103/50000: episode: 19, duration: 7.540s, episode steps:  28, steps per second:   4, episode reward:  2.700, mean reward:  0.096 [ 0.000,  0.100], mean action: 1.286 [0.000, 3.000],  loss: 0.077055, mae: 0.362684, mean_q: 0.635730
   123/50000: episode: 20, duration: 0.493s, episode steps:  20, steps per second:  41, episode reward:  2.000, mean reward:  0.100 [ 0.000,  0.200], mean action: 1.150 [0.000, 3.000],  loss: 0.036437, mae: 0.276040, mean_q: 0.464634
   131/50000: episode: 21, duration: 0.209s, episode steps:   8, steps per second:  38, episode reward:  0.800, mean reward:  0.100 [ 0.000,  0.200], mean action: 2.000 [0.000, 3.000],  loss: 0.012524, mae: 0.214168, mean_q: 0.357289
   135/50000: episode: 22, duration: 0.117s, episode steps:   4, steps per second:  34, episode reward:  0.300, mean reward:  0.075 [ 0.000,  0.100], mean action: 1.750 [1.000, 3.000],  loss: 0.006998, mae: 0.179702, mean_q: 0.319817
   145/50000: episode: 23, duration: 0.254s, episode steps:  10,

In [8]:
scores = dqn.test(env, nb_episodes = 100, visualize = False)
dqn.save_weights('Large_weights.h5',overwrite = True)

Testing for 100 episodes ...
Episode 1: reward: 0.100, steps: 2
Episode 2: reward: 1.700, steps: 18
Episode 3: reward: 0.100, steps: 2
Episode 4: reward: 0.900, steps: 10
Episode 5: reward: 0.200, steps: 3
Episode 6: reward: 0.200, steps: 3
Episode 7: reward: 1.700, steps: 18
Episode 8: reward: 34.500, steps: 16
Episode 9: reward: 0.800, steps: 9
Episode 10: reward: 2.500, steps: 25
Episode 11: reward: 3.500, steps: 34
Episode 12: reward: 0.300, steps: 4
Episode 13: reward: 0.200, steps: 3
Episode 14: reward: 0.400, steps: 5
Episode 15: reward: 0.400, steps: 5
Episode 16: reward: 0.400, steps: 5
Episode 17: reward: 0.600, steps: 7
Episode 18: reward: 1.900, steps: 19
Episode 19: reward: 0.100, steps: 2
Episode 20: reward: 0.400, steps: 4
Episode 21: reward: 1.400, steps: 15
Episode 22: reward: 0.100, steps: 2
Episode 23: reward: 0.100, steps: 2
Episode 24: reward: 0.900, steps: 10
Episode 25: reward: 0.100, steps: 2
Episode 26: reward: 1.600, steps: 17
Episode 27: reward: 34.100, steps